## O caixeiro viajante que gosta de viajar  
O caixeiro com gasolina infinita (e sem conciência ambiental)


## Introdução



Este problema é a história de um caixeiro que viaja entre cidade para vender sua mercadoria. Nesta variedade, o caixeiro possue gasolina infinita e gosta muito de estrada, portando, o objetivo é maximizar o trajeto dentro de um conjunto de cidades, para que assim a distância percorrida seja a maior possível.


## Objetivo



Encontre o caminho de *maior* distância no problema do caixeiro viajante e mostre ele de forma gráfica.

**Considerações do experimento**: Considere um número $n\geq 7$ de coordenadas $(x,y)$ de cidades e que o caixeiro tenha combustível infinito. Você pode gerar as coordenadas de forma aleatória ou simplesmente usar as coordenadas que desejar. O caixeiro só anda em linha reta e apenas entre duas cidades. O caixeiro começa e termina seu trajeto na mesma cidade e, fora a cidade inicial, ele não visita nenhuma outra cidade mais de uma vez.

## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [ ]:
from funcoes import cria_cidades
from funcoes import populacao_inicial_cv
from funcoes import funcao_objetivo_cv
from funcoes import funcao_objetivo_pop_cv
from funcoes import selecao_torneio_max #nova funcao
from funcoes import cruzamento_ordenado
from funcoes import mutacao_de_troca

## Códigos e discussão



-   Use células de código para o código.

-   Use células de texto para a discussão.

-   A discussão não deve ser feita em comentários dentro das células de código. Toda discussão deve acontecer após o resultado sendo discutido foi apresentado. Exemplo: não discuta um gráfico antes de apresentá-lo.



In [ ]:
### Constantes 


# relacionadas a busca
TAMANHO_POP = 10
NUM_GERACOES = 50
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 2

# relacionadas ao problema a ser resolvido
NUMERO_DE_CIDADES = 9
CIDADES = cria_cidades(NUMERO_DE_CIDADES)

In [ ]:
# criar população 
populacao = populacao_inicial_cv(TAMANHO_POP, CIDADES)

# fitness da população 

melhor_fitness = float("-inf") # Nesta linha está definido o parametro inicial para o fitness
                               # Como este é um problema de maximização, começamos com um valor
                               # extremamente baixo, assim qualquer valor acima disto será considerado
    
# cruzamento 

for i in (NUM_GERACOES):
    
    # 


## Conclusão



Delete este texto e escreva sua conclusão.



## Referências consultadas



1.  Delete este texto e inclua suas referências ordenadas numericamente. Se for referenciar no notebook, use o número entre colchetes (exemplo: para citar essa referência aqui escreva &ldquo;[1]&rdquo; sem as áspas).

2.  Cada item deve ser numerado. Siga o padrão apresentado.

3.  Caso não tenha nenhuma referência consultada, delete esta seção e o texto contido nela!



## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.

